## **ANALYTICAL FORWARD AND INVERSE KINEMATICS**

**Importing necessary libraries**

In [2]:
from spatialmath import SE3
from spatialmath.base import *
import roboticstoolbox as rtb
import numpy as np
import sympy as sym

In [3]:
# Define joint variables using SymPy
theta_1 = sym.Symbol('theta_1')
theta_2 = sym.Symbol('theta_2')
d_3 = sym.Symbol('d_3')
theta_4 = sym.Symbol('theta_4')
theta_5 = sym.Symbol('theta_5')
theta_6 = sym.Symbol('theta_6')

**Custom Stanford Robot Class with Standard DH Parameters and Joint Limits**

In [5]:
class StanfordG2(rtb.DHRobot):
    def __init__(self):
        super().__init__(
            [
                rtb.RevoluteDH(d=0.412, a=0, alpha=-np.pi/2),
                rtb.RevoluteDH(d=0.154, a=0, alpha=np.pi/2),
                rtb.PrismaticDH(theta=0, a=0, alpha=0, qlim=[0.1, 1.0]),  # Setting limits for the prismatic joint
                rtb.RevoluteDH(a=0, alpha=-np.pi/2),
                rtb.RevoluteDH(a=0, alpha=np.pi/2),
                rtb.RevoluteDH(d=0, a=0, alpha=0),
            ], name="StanfordG2"
        )

    def ikine(self, T, q0=None, **kwargs):
        """
        Compute the inverse kinematics for the Stanford robot.

        :param T: Desired end-effector pose (SE3 object).
        :param q0: Initial guess for joint coordinates.
        :return: Named tuple containing the solution.
        """
        sol = self.ikine_LM(T, q0=q0, **kwargs)
        return sol

In [6]:
stanford_g2_robot = StanfordG2()
stanford_g2_robot

DHRobot: StanfordG2, 6 joints (RRPRRR), dynamics, standard DH parameters
┌─────┬───────┬────┬────────┬─────────┬────────┐
│ θⱼ  │  dⱼ   │ aⱼ │   ⍺ⱼ   │   q⁻    │   q⁺   │
├─────┼───────┼────┼────────┼─────────┼────────┤
│ q1  │ 0.412 │  0 │ -90.0° │ -180.0° │ 180.0° │
│ q2  │ 0.154 │  0 │  90.0° │ -180.0° │ 180.0° │
│0.0° │    q3 │  0 │   0.0° │     0.1 │    1.0 │
│ q4  │     0 │  0 │ -90.0° │ -180.0° │ 180.0° │
│ q5  │     0 │  0 │  90.0° │ -180.0° │ 180.0° │
│ q6  │     0 │  0 │   0.0° │ -180.0° │ 180.0° │
└─────┴───────┴────┴────────┴─────────┴────────┘

┌─┬──┐
└─┴──┘

In [7]:
T= stanford_g2_robot.fkine([10,15,3,20,30,40])
sym.simplify(T.A)

[[0.0135032803477586, 0.554769938744441, -0.831894209912137, -1.5531347860644], [-0.241696844416887, 0.809110022868043, 0.535652505168597, -1.19052795507814], [0.970257850686491, 0.193833139478172, 0.14501178304285, -1.86706373857646], [0.0, 0.0, 0.0, 1.0]]

In [8]:
T = stanford_g2_robot.fkine([theta_1, theta_2, d_3, theta_4, theta_5, theta_6])
T_simp = sym.simplify(T.A)
T_simp

[[-((1.0*sin(theta_1)*sin(theta_4) + 5.55111512312578e-17*sin(theta_2)*sin(theta_4)*cos(theta_1) - 1.0*cos(theta_1)*cos(theta_2)*cos(theta_4))*cos(theta_5) + (6.12323399573677e-17*(3.74939945665464e-33*sin(theta_1)*cos(theta_2) + 1.0*sin(theta_1) + 6.12323399573677e-17*sin(theta_2)*cos(theta_1))*cos(theta_4) + 6.12323399573677e-17*sin(theta_1)*cos(theta_2) - 6.12323399573677e-17*sin(theta_1) + 1.0*sin(theta_2)*cos(theta_1) + 1.53080849893419e-17*sin(-theta_1 + theta_2 + theta_4) + 1.53080849893419e-17*sin(theta_1 - theta_2 + theta_4) - 1.53080849893419e-17*sin(theta_1 + theta_2 - theta_4) + 1.53080849893419e-17*sin(theta_1 + theta_2 + theta_4))*sin(theta_5))*cos(theta_6) - (-6.12323399573677e-17*(1.0*sin(theta_1)*sin(theta_4) + 5.55111512312578e-17*sin(theta_2)*sin(theta_4)*cos(theta_1) - 1.0*cos(theta_1)*cos(theta_2)*cos(theta_4))*sin(theta_5) + 1.0*(3.74939945665464e-33*sin(theta_1)*cos(theta_2) + 1.0*sin(theta_1) + 6.12323399573677e-17*sin(theta_2)*cos(theta_1))*cos(theta_4) + 6.123

In [9]:
desired_pose = SE3(0.7, 0.2, 0.1) * SE3.OA([0, 1, 0], [0, 0, -1])

In [10]:
sol = stanford_g2_robot.ikine(desired_pose)
print("Joint coordinates: ", sol.q)

Joint coordinates:  [-2.65014768 -1.98403559  0.77693485 -3.1415924   1.15755691  0.49144481]


In [11]:
# Verify the solution by computing the FK of the obtained joint coordinates
computed_pose = stanford_g2_robot.fkine(sol.q)
print(computed_pose)
print("Residual: ", np.linalg.norm(desired_pose.A - computed_pose.A))

  -1        -6.197e-08  3.66e-08  0.7       
  -6.197e-08  1         2.779e-07  0.2       
  -3.66e-08  2.779e-07 -1         0.1       
   0         0         0         1         

Residual:  4.674594933520768e-07
